# Working with Simulation Directories

Analyzing simulation data is as important as running the simulation itself. In this notebook, we will see how we easily access all the data using the [SimDir](../simdir.rst) module.

(This notebook is meant to be converted in Sphinx documentation and not used directly.)

In [1]:
from kuibit import simdir as sd

Loading simulation data is as easy as generating `SimDir` objects. Just provide the path, and kuibit will do the rest.

In [2]:
sim = sd.SimDir("../../tests/tov")

We can get an overview of what is available by printing `sim`:

In [3]:
print(sim)

Indexed 446 files and 5 subdirectories
Folder /home/runner/work/kuibit/kuibit/tests/tov
Available scalar timeseries:
['physical_time_per_hour', 'current_physical_time_per_hour', 'time_total', 'time_evolution', 'time_computing', 'time_communicating', 'time_io', 'evolution_steps_count', 'local_grid_points_per_second', 'total_grid_points_per_second', 'local_grid_point_updates_count', 'total_grid_point_updates_count', 'local_interior_points_per_second', 'total_interior_points_per_second', 'local_interior_point_updates_count', 'total_interior_point_updates_count', 'io_per_second', 'io_bytes_per_second', 'io_bytes_ascii_per_second', 'io_bytes_binary_per_second', 'io_count', 'io_bytes_count', 'io_bytes_ascii_count', 'io_bytes_binary_count', 'comm_per_second', 'comm_bytes_per_second', 'comm_count', 'comm_bytes_count', 'time_levels']

Available minimum timeseries:
['vel[0]', 'vel[1]', 'vel[2]', 'kxx', 'kxy', 'kxz', 'kyy', 'kyz', 'kzz', 'eps', 'gxx', 'gxy', 'gxz', 'gyy', 'gyz', 'gzz', 'M1', 'M2'

This is a long output because we have a lot of available data! Everything that is printed is the output is easily accessible through `SimDir`. Moreover, the loading of data is lazy and the result is cached: computations are done only when needed, and all the subsequent calls will be immediate.

If you have thorns with custom output, you have to extend `kuibit` to analyze them. `SimDir` has multiple submodules to handle the different kind of data. Let us explore them.

## SimDir and pickles

`SimDir` performs some operations only when they needed. This work has be done every time the Python interpreter is killed. In some cases, it is useful to save the progress to files, resulting in faster execution times. This can be done with pickles. The simplest way to do so is as follows.

In [4]:
with sd.SimDir("../../tests/tov", pickle_file="/tmp/sim.pickle") as sim2:
    print(sim2)

Indexed 446 files and 5 subdirectories
Folder /home/runner/work/kuibit/kuibit/tests/tov
Available scalar timeseries:
['physical_time_per_hour', 'current_physical_time_per_hour', 'time_total', 'time_evolution', 'time_computing', 'time_communicating', 'time_io', 'evolution_steps_count', 'local_grid_points_per_second', 'total_grid_points_per_second', 'local_grid_point_updates_count', 'total_grid_point_updates_count', 'local_interior_points_per_second', 'total_interior_points_per_second', 'local_interior_point_updates_count', 'total_interior_point_updates_count', 'io_per_second', 'io_bytes_per_second', 'io_bytes_ascii_per_second', 'io_bytes_binary_per_second', 'io_count', 'io_bytes_count', 'io_bytes_ascii_count', 'io_bytes_binary_count', 'comm_per_second', 'comm_bytes_per_second', 'comm_count', 'comm_bytes_count', 'time_levels']

Available minimum timeseries:
['vel[0]', 'vel[1]', 'vel[2]', 'kxx', 'kxy', 'kxz', 'kyy', 'kyz', 'kzz', 'eps', 'gxx', 'gxy', 'gxz', 'gyy', 'gyz', 'gzz', 'M1', 'M2'

When this code is run, `kuibit` checks if a pickle file exists in `/tmp/sim.pickle`. If yes, it will load it, effectively "resuming a previous session". At the end of the `with` block, the file is updated with the new work done. If the file did not exist initially, it will be created.

It is important to be careful when working with pickles as the risk of stale objects is high. If the data is changing (e.g., new restarts are added), it is likely that it will result in errors. 

It is always possible to save the state of a `SimDir` using the `save` method and reads back the data with the `load_SimDir` function

In [5]:
sim.save("/tmp/saved.pickle") # The filename can be anything
sim3 = sd.load_SimDir("/tmp/saved.pickle")

## Cactus_scalars
The [cactus_scalars](../cactus_scalars.rst) module understands ASCII files as output by CarpetASCII. It supports all the reductions (maximum, minimum, ...), 0d files, as well as scalar data. Using regular expression on the header of the file, `cactus_scalars` can process one variable per file and multiple variables per file. In `SimDir`, you can access scalars with the `ts` or `timeseries` attributes. 

In [6]:
timeseries = sim.ts
# equivalent: timeseries = sim.timeseries

If we printed `timeseries`, we would find part of the output from `SimDir`. This is because the printed output from `SimDir` is obtained printing all the different submodules. Another way to see the available variable for a secific reduction is with the `keys` method:

In [7]:
timeseries.maximum.keys()

dict_keys(['vel[0]', 'kxx', 'kxy', 'kxz', 'kyy', 'kyz', 'kzz', 'M1', 'M2', 'M3', 'press', 'alp', 'gxx', 'gxy', 'gxz', 'gyy', 'gyz', 'gzz', 'eps', 'rho', 'vel[1]', 'vel[2]', 'H'])

There are equivalent three different way to access those variables as [timeseries](../series.rst).

In [8]:
rho_max1 = timeseries.maximum.fields.rho
rho_max2 = timeseries.maximum['rho']
rho_max3 = timeseries.maximum.get('rho')

These objects are [timeseries](../series.rst), which are extremely convienent representation of data evolving in time. These series span multiple simulation restarts and have files in different folders: `kuibit` does all the hard work to find the files and combine them. Moreover, they `timeseries` have a lot of features, as you can see in the [timeseries tutorial](timeseries.ipynb).